<div class="alert alert-block alert-info">
<b>Vaja: </b> 
Uvozite dataset <i>cancer.csv</i>, ki vsebuje podatke o obolelosti z rakom, smrti zaradi raka in socialno-ekonomskem stanju prebivalcev okrožji Združenih držav amerike.
    
Ustvarite LinearRegression model, ki bo napovedoval umrljivost ("TARGET_deathRate").
    
Za primerjavo - naš najboljši model je imel sledeče karakteristike:
    
    RMSE:        22.06
    KFold:       18
</div>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Vnos podatkov:

cancer = pd.read_csv("data/cancer.csv")

In [4]:
cancer.head()

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,...,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",39.3,...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",45.0,...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


In [5]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              3047 non-null   float64
 1   avgDeathsPerYear         3047 non-null   int64  
 2   TARGET_deathRate         3047 non-null   float64
 3   incidenceRate            3047 non-null   float64
 4   medIncome                3047 non-null   int64  
 5   popEst2015               3047 non-null   int64  
 6   povertyPercent           3047 non-null   float64
 7   studyPerCap              3047 non-null   float64
 8   binnedInc                3047 non-null   object 
 9   MedianAge                3047 non-null   float64
 10  MedianAgeMale            3047 non-null   float64
 11  MedianAgeFemale          3047 non-null   float64
 12  Geography                3047 non-null   object 
 13  AvgHouseholdSize         3047 non-null   float64
 14  PercentMarried          

In [6]:
# Preverimo kateri stolpci imajo manjkajoče vrednosti:

cancer.isna().sum()

avgAnnCount                   0
avgDeathsPerYear              0
TARGET_deathRate              0
incidenceRate                 0
medIncome                     0
popEst2015                    0
povertyPercent                0
studyPerCap                   0
binnedInc                     0
MedianAge                     0
MedianAgeMale                 0
MedianAgeFemale               0
Geography                     0
AvgHouseholdSize              0
PercentMarried                0
PctNoHS18_24                  0
PctHS18_24                    0
PctSomeCol18_24            2285
PctBachDeg18_24               0
PctHS25_Over                  0
PctBachDeg25_Over             0
PctEmployed16_Over          152
PctUnemployed16_Over          0
PctPrivateCoverage            0
PctPrivateCoverageAlone     609
PctEmpPrivCoverage            0
PctPublicCoverage             0
PctPublicCoverageAlone        0
PctWhite                      0
PctBlack                      0
PctAsian                      0
PctOther

In [7]:
# Manjkajoče vrednosti stolpca PctEmployed napolnimo z najbolj pogosto vrednostjo (kot na predavanjih)

cancer["PctEmployed16_Over"] = cancer["PctEmployed16_Over"].fillna(cancer["PctEmployed16_Over"].median())

In [8]:
cancer["binnedInc"].head()

0     (61494.5, 125635]
1    (48021.6, 51046.4]
2    (48021.6, 51046.4]
3      (42724.4, 45201]
4    (48021.6, 51046.4]
Name: binnedInc, dtype: object

In [9]:
# Čiščenje podatkov stolpca:

cancer["binnedInc"]=cancer["binnedInc"].str.replace("(","")
cancer["binnedInc"]=cancer["binnedInc"].str.replace("[","")
cancer["binnedInc"]=cancer["binnedInc"].str.replace("]","")

In [10]:
# Iz obeh vrednosti stolpca izračunamo sredino:

x = cancer["binnedInc"].str.split(",", expand=True).astype(float)
y = (x[0] + x[1]) / 2
cancer["binnedInc"] = y

In [11]:
cancer["binnedInc"].head()

0    93564.75
1    49534.00
2    49534.00
3    43962.70
4    49534.00
Name: binnedInc, dtype: float64

In [12]:
cancer.nunique()

avgAnnCount                 929
avgDeathsPerYear            608
TARGET_deathRate           1053
incidenceRate              1506
medIncome                  2920
popEst2015                 2999
povertyPercent              333
studyPerCap                1117
binnedInc                    10
MedianAge                   325
MedianAgeMale               298
MedianAgeFemale             296
Geography                  3047
AvgHouseholdSize            199
PercentMarried              362
PctNoHS18_24                405
PctHS18_24                  469
PctSomeCol18_24             343
PctBachDeg18_24             219
PctHS25_Over                361
PctBachDeg25_Over           281
PctEmployed16_Over          409
PctUnemployed16_Over        195
PctPrivateCoverage          498
PctPrivateCoverageAlone     459
PctEmpPrivCoverage          450
PctPublicCoverage           395
PctPublicCoverageAlone      319
PctWhite                   3044
PctBlack                   2972
PctAsian                   2852
PctOther

In [13]:
# Odstranimo Geography stolpec:

cancer.drop("Geography", axis=1, inplace=True)

In [14]:
cancer.head()

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,...,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,93564.75,39.3,...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,49534.00,33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,49534.00,45.0,...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,43962.70,42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,49534.00,48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


In [15]:
Predictors = ["PctBachDeg25_Over", 
              "incidenceRate", 
              "PctPublicCoverageAlone", 
              "povertyPercent", 
              "medIncome", 
              "PctEmployed16_Over", 
              "PctHS25_Over", 
              "PctPublicCoverage", 
              "PctPrivateCoverage", 
              "PctUnemployed16_Over", 
              "PctMarriedHouseholds", 
              "PctBachDeg18_24", 
              "PctEmpPrivCoverage", 
              "PercentMarried", 
              "PctHS18_24", 
              "PctBlack", 
              "binnedInc"]

Target = ["TARGET_deathRate"]

X = cancer[Predictors].values
y = cancer[Target].values

In [41]:
# Dataset razdelimo v train in test del. Izbral sem test size 20%.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

lr = LinearRegression()
lrRegModel = lr.fit(X_train, y_train)
lrPrediction = lrRegModel.predict(X_test)

print("R2 value", r2_score(y_train, lrRegModel.predict(X_train)))
print("Accuracy", 100-(np.mean((np.abs(y_test-lrPrediction)/y_test))*100))
print("RMSE", mean_squared_error(y_test, lrPrediction, squared=False))

TestingData = pd.DataFrame(X_test, columns=Predictors)
TestingData["Target"] = y_test
TestingData["PredictedValue"] = lrPrediction
TestingData["APE"] = (np.abs(y_test-lrPrediction)/y_test)*100
TestingData.head()

R2 value 0.5056375733672549
Accuracy 91.08516048837879
RMSE 20.59413477050582


,PctBachDeg25_Over,incidenceRate,PctPublicCoverageAlone,povertyPercent,medIncome,PctEmployed16_Over,PctHS25_Over,PctPublicCoverage,PctPrivateCoverage,PctUnemployed16_Over,PctMarriedHouseholds,PctBachDeg18_24,PctEmpPrivCoverage,PercentMarried,PctHS18_24,PctBlack,binnedInc,Target,PredictedValue,APE
0,12.0,477.4,22.7,19.1,44358.0,51.9,31.0,41.4,62.3,10.6,51.807095,3.7,40.2,52.9,34.5,0.437922,43962.70,186.5,186.172781,0.175453
1,22.3,450.2,13.5,22.6,47966.0,59.6,22.8,24.6,77.3,6.5,39.728702,14.3,54.5,39.3,14.3,12.588086,46611.30,152.5,163.218112,7.028270
2,23.0,508.7,11.2,7.2,71885.0,62.8,25.9,27.1,82.6,6.0,52.770765,13.5,60.9,54.0,26.0,1.611447,93564.75,174.2,166.057375,4.674297
3,6.8,484.4,21.0,15.7,37961.0,36.1,41.9,53.1,60.8,16.0,49.213759,3.5,27.5,51.5,26.3,0.361664,38888.25,207.6,192.693059,7.180607
4,11.5,457.7,18.3,13.0,49698.0,59.0,35.0,28.5,62.1,10.1,55.195246,2.1,46.4,52.0,38.3,11.578626,49534.00,192.0,183.562806,4.394372


In [43]:
# Probam še test size 30%.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

lr = LinearRegression()
lrRegModel = lr.fit(X_train, y_train)
lrPrediction = lrRegModel.predict(X_test)

print("R2 value", r2_score(y_train, lrRegModel.predict(X_train)))
print("Accuracy", 100-(np.mean((np.abs(y_test-lrPrediction)/y_test))*100))
print("RMSE", mean_squared_error(y_test, lrPrediction, squared=False))

TestingData = pd.DataFrame(X_test, columns=Predictors)
TestingData["Target"] = y_test
TestingData["PredictedValue"] = lrPrediction
TestingData["APE"] = (np.abs(y_test-lrPrediction)/y_test)*100
TestingData.head()

R2 value 0.5160155124813814
Accuracy 90.83546844035453
RMSE 20.978520088872042


,PctBachDeg25_Over,incidenceRate,PctPublicCoverageAlone,povertyPercent,medIncome,PctEmployed16_Over,PctHS25_Over,PctPublicCoverage,PctPrivateCoverage,PctUnemployed16_Over,PctMarriedHouseholds,PctBachDeg18_24,PctEmpPrivCoverage,PercentMarried,PctHS18_24,PctBlack,binnedInc,Target,PredictedValue,APE
0,12.0,477.4,22.7,19.1,44358.0,51.9,31.0,41.4,62.3,10.6,51.807095,3.7,40.2,52.9,34.5,0.437922,43962.70,186.5,185.746518,0.404012
1,22.3,450.2,13.5,22.6,47966.0,59.6,22.8,24.6,77.3,6.5,39.728702,14.3,54.5,39.3,14.3,12.588086,46611.30,152.5,162.608695,6.628652
2,23.0,508.7,11.2,7.2,71885.0,62.8,25.9,27.1,82.6,6.0,52.770765,13.5,60.9,54.0,26.0,1.611447,93564.75,174.2,166.954098,4.159530
3,6.8,484.4,21.0,15.7,37961.0,36.1,41.9,53.1,60.8,16.0,49.213759,3.5,27.5,51.5,26.3,0.361664,38888.25,207.6,193.875187,6.611182
4,11.5,457.7,18.3,13.0,49698.0,59.0,35.0,28.5,62.1,10.1,55.195246,2.1,46.4,52.0,38.3,11.578626,49534.00,192.0,182.828445,4.776852
